# Importing Libraries

In [1]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
import datetime
import zipfile
import pandas as pd
import requests
from openpyxl import * 
import os

# Chromedriver Function

In [2]:
#Chromedriver is the name of the function

def ChromeDriver(url): #Can add multiple urls! Note in Python do not need to define datatypes
    
    #indentation signfies inside a function
    chrome_path= "/Users/myaturner/chromedriver"
    
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized");
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    
    driver = webdriver.Chrome(executable_path=chrome_path, options=options)
    driver.get(url)
    
    return driver #ends the function

# Using Selenium to navigate to the Covid Summary for Maine

In [3]:
link = 'https://www.maine.gov/dhhs/mecdc/infectious-disease/epi/airborne/coronavirus/data.shtml'
driver = ChromeDriver(link)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#Copy Full XPath
Find_by_XPath = driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div[3]/p[6]/a')

time.sleep(4)

Find_by_XPath[0].click()

# Check: Printing the Title of website

In [4]:
driver = ChromeDriver(link)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

Find_by_XPath = driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div[3]/h1')

time.sleep(6)

print(Find_by_XPath[0].text)

COVID-19: Maine Data


# Check: Printing the date of website

In [5]:
driver = ChromeDriver(link)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

Find_by_SoupClass = soup.find(class_="crumb_trail")

time.sleep(6)

print(Find_by_SoupClass.text)



Sun 20 Dec 2020
      
      JavaScript that displays the current date.
      

DHHS → MeCDC → Disease Surveillance → Epidemiology → Airborne and Direct Contact Diseases → Coronavirus → Maine Data 



# Acessing the Most Recent Maine Covid Summary

In [6]:
#Accessing the latest data for cases by race
#source(s): https://datatofish.com/read_excel/, https://riptutorial.com/pandas/example/5927/read-a-specific-sheet

cases_by_race = pd.read_excel(r'/Users/myaturner/Downloads/Maine_COVID19_Summary.xlsx', 'cases_by_race')
cases_by_race

,PATIENT_RACE_CALC,DATA_REFRESH_DT,DATA_AS_OF_DT,CASES
0,NaN,2020-12-20,2020-12-19,7
1,American Indian or Alaska Native,2020-12-20,2020-12-19,59
2,Asian,2020-12-20,2020-12-19,252
3,Black or African American,2020-12-20,2020-12-19,1465
4,Native Hawaiian or Other Pacific Islander,2020-12-20,2020-12-19,10
5,Not disclosed,2020-12-20,2020-12-19,1925
6,Other Race,2020-12-20,2020-12-19,418
7,Two or more,2020-12-20,2020-12-19,158
8,White,2020-12-20,2020-12-19,14652


In [7]:
#Acessing the latest data for cases by ethnicity

cases_by_ethnicity = pd.read_excel(r'/Users/myaturner/Downloads/Maine_COVID19_Summary.xlsx', 'cases_by_ethnicity')
cases_by_ethnicity

,PATIENT_ETHNICITY,DATA_REFRESH_DT,DATA_AS_OF_DT,CASES
0,unknown,2020-12-20,2020-12-19,4140
1,Not Hispanic or Latino,2020-12-20,2020-12-19,14416
2,Hispanic or Latino,2020-12-20,2020-12-19,390


# Appending the latest data to old data

In [8]:
#Appending Race Data

maine_covid_summary_previous_race = pd.read_excel('maine_covid_summary.xlsx', sheet_name = 'cases_by_race')

maine_covid_summary_latest_race = pd.concat([maine_covid_summary_previous_race, cases_by_race])

#Dropping unamed columns

maine_covid_summary_latest_race = maine_covid_summary_latest_race.drop(columns=['Unnamed: 0'])
#maine_covid_summary_latest_race

In [9]:
#Appending Ethnicity Data
maine_covid_summary_previous_ethnicity = pd.read_excel('maine_covid_summary.xlsx', sheet_name = 'cases_by_ethnicity')

maine_covid_summary_latest_ethnicity = pd.concat([maine_covid_summary_previous_ethnicity, cases_by_ethnicity])

#Dropping unamed columns
maine_covid_summary_latest_ethnicity = maine_covid_summary_latest_ethnicity.drop(columns=['Unnamed: 0'])

#maine_covid_summary_latest_ethnicity

# Writing the latest data to a Spreadsheet

In [10]:
from pandas import ExcelWriter
from pandas import ExcelFile

with ExcelWriter('maine_covid_summary.xlsx') as writer:
    
    #Writing Cases by Race to a local Spreadsheet
    maine_covid_summary_latest_race.to_excel(writer,'cases_by_race')
    
    #Writing Cases by Ethnicity to a local Spreadsheet
    maine_covid_summary_latest_ethnicity.to_excel(writer,'cases_by_ethnicity')


#If prints out this message, data was successfully saved.
#writer.save()
print('Dataframe is written successfully to Excel Sheet.')

Dataframe is written successfully to Excel Sheet.


# Removing Old Maine Covid Summary

In [11]:
#In order to run the scraper another day, need to remove the old data so it does not confuse the algorithm
#source(s): https://www.dummies.com/programming/python/how-to-delete-a-file-in-python/

os.remove(r'/Users/myaturner/Downloads/Maine_COVID19_Summary.xlsx')
print("File Removed!")

File Removed!
